## This is a tutorial notebook for producing plots after running any of the pipelines in the repository. First, run one of the pipelines using one of the provided yaml files (or a similar one). Then run this notebook to make plots and interpret the outputs. You only need to modify the first code cell in the notebook.

In [ ]:
## ONLY NEED TO MODIFY VARIABLES IN THIS CELL ##

#yaml file used when running the pipeline
yaml_file = '../needlet_ILC_pipeline/example_yaml_files/laptop.yaml' 

#pipeline that was run (either 'multifrequency', 'HILC', or 'NILC')
pipeline = 'NILC'

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import sys
from matplotlib.ticker import FormatStrFormatter
from getdist import plots, MCSamples
import getdist
sys.path.append('../shared')
from utils import GaussianNeedlets, get_naming_str

In [ ]:
if pipeline == 'multifrequency':
    sys.path.append('../multifrequency_pipeline')
elif pipeline == 'NILC':
    sys.path.append('../needlet_ILC_pipeline')
elif pipeline == 'HILC':
    sys.path.append('../harmonic_ILC_pipeline')
from input import Info

In [ ]:
font = {'size'   : 20, 'family':'STIXGeneral'}
plt.rcParams.update({
     'text.usetex': True,
     'font.family': 'serif',
     'font.sans-serif': ['Computer Modern'],
     'font.size':20})
plt.rc_context({'axes.autolimit_mode': 'round_numbers'})

getdist.plots.GetDistPlotSettings.legend_fontsize = 14

In [ ]:
# read in the input file and set up relevant info object
inp = Info(yaml_file)
inp.ell_sum_max = inp.ellmax

ells = np.arange(inp.ellmax+1)
res = stats.binned_statistic(ells[2:], ells[2:], statistic='mean', bins=inp.Nbins)
mean_ells = np.round((res[1][:-1]+res[1][1:])/2)

naming_str = get_naming_str(inp, pipeline)

# Plot posteriors

In [ ]:
acmb_arr = pickle.load(open(f'{inp.output_dir}/posteriors/acmb_array_{naming_str}.p', 'rb'))
atsz_arr = pickle.load(open(f'{inp.output_dir}/posteriors/atsz_array_{naming_str}.p', 'rb'))

In [ ]:
names = [r'$A_{\mathrm{CMB}}$', r'$A_{\mathrm{ftSZ}}$']
labels =  names
samples = MCSamples(samples=[acmb_arr,atsz_arr], names = names, labels = labels)

g = plots.get_subplot_plotter()
g.triangle_plot([samples], filled=True)

In [ ]:
for par in [r"$A_{\mathrm{CMB}}$", r'$A_{\mathrm{ftSZ}}$']:
    print(samples.getInlineLatex(par,limit=1))

# Plot Power Spectra and Covariance and Correlation Matrices

### (If using LFI, plot these matrices using the simulations generated at the fiducial parameter values).

In [ ]:
from matplotlib.colors import SymLogNorm, LogNorm

## Compute PS and Cov

In [ ]:
def get_PS_array(inp, Clpq, pipeline):
    '''
    ARGUMENTS
    ---------
    inp: Info object containing input paramter specifications
    Clpq: arrays of varying dimensions based on pipeline (Clij for multifrequency)
    pipeline: str, 'NILC', 'HILC', or 'multifrequency'
    
    RETURNS
    -------
    PS: power spectrum array of shape (Nsims, 2, 2, Nbins), 
        where 2 is for N_preserved_comps in HILC or NILC and for Nfreqs in multifrequency
    '''
    if inp.use_lfi:
        PS = Clpq
    elif (pipeline == 'NILC') or (pipeline == 'HILC' and inp.use_symbolic_regression):
        PS = Clpq[:,0,0,0]
    elif (pipeline == 'HILC' and not inp.use_symbolic_regression) or (pipeline == 'multifrequency'):
        PS = Clpq[:,:,:,0]
    return PS
    

In [ ]:
def get_PScov_sim(inp, PS):
    '''
    ARGUMENTS
    ---------
    inp: Info object containing input paramter specifications
    PS: For NILC or HILC, PS is Clpq of shape (Nsims, N_preserved_comps=2, N_preserved_comps=2, Nbins) ndarray 
        containing propagation of each pair of component maps to HILC or NILC map auto- and cross-spectra.
        For multifrequency, PS is Clij of shape (Nsims, Nfreqs=2, Nfreqs=2, Nbins).
    
    RETURNS
    -------
    cov: (3*Nbins, 3*Nbins) ndarray containing covariance matrix Cov_{pq,rs} (Cov_{ij,kl} for multifrequency)
        index as cov[(0-2 for ClTT ClTy Clyy)*Nbins + bin1, (0-2 for ClTT ClTy Clyy)*Nbins + bin2] if HILC or NILC
        index as cov[(0-2 for Cl00 Cl01 Cl11)*Nbins + bin1, (0-2 for Cl00 Cl01 Cl11)*Nbins + bin2] if multifrequency
    '''
    Clpq_tmp = PS
    Clpq_tmp = np.array([Clpq_tmp[:,0,0], Clpq_tmp[:,0,1], Clpq_tmp[:,1,1]])
    Clpq_tmp = np.transpose(Clpq_tmp, axes=(0,2,1)) #shape (3, Nbins, Nsims)
    Clpq_tmp = np.reshape(Clpq_tmp, (inp.Nbins*3, -1))
    cov = np.cov(Clpq_tmp)
    return cov

In [ ]:
def plot_cov_matrix(inp, pipeline, PS, title=None, corr=False, colorbar=False):
    '''
    ARGUMENTS
    ---------
    inp: Info object containing input paramter specifications
    pipeline: str, 'NILC', 'HILC', or 'multifrequency'
    PS: For NILC or HILC, PS is Clpq of shape (Nsims, N_preserved_comps=2, N_preserved_comps=2, Nbins) ndarray 
        containing propagation of each pair of component maps to HILC or NILC map auto- and cross-spectra.
        For multifrequency, PS is Clij of shape (Nsims, Nfreqs=2, Nfreqs=2, Nbins).
    title: str, title of plot
    corr: Bool, if True plots correlation matrix, if False plots absolute covariance matrix
    colorbar: Bool, whether to include colorbar in the plot
    
    RETURNS
    -------
    None
    '''
    PScov = get_PScov_sim(inp, PS)   
    if not corr:
        plt.imshow(np.abs(PScov), norm=LogNorm())
    else:
        diag = np.diag(PScov)
        PScorr = np.einsum('ab,a,b->ab', PScov, 1/np.sqrt(diag), 1/np.sqrt(diag))
        plt.imshow(PScorr, vmin=-1., vmax=1.)
    ax = plt.gca()
    ax.set_xticks([5,15,25])
    ax.set_yticks([5,15,25])
    if pipeline == 'multifrequency':
        labels = [r'$C_\ell^{00}$', r'$C_\ell^{01}$', r'$C_\ell^{11}$']
    else:
        labels = [r'$C_\ell^{\hat{T}\hat{T}}$', r'$C_\ell^{\hat{T}\hat{y}}$', r'$C_\ell^{\hat{y}\hat{y}}$']
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
    if colorbar:
        plt.colorbar()
    if title:
        plt.title(title, fontsize=15)

In [ ]:
if pipeline == 'multifrequency':
    Clij = pickle.load(open(f'{inp.output_dir}/data_vecs/Clij_{naming_str}.p', 'rb'))
    PS = get_PS_array(inp, Clij, pipeline)
else:
    Clpq = pickle.load(open(f'{inp.output_dir}/data_vecs/Clpq_{naming_str}.p', 'rb'))
    PS = get_PS_array(inp, Clpq, pipeline)
PScov = get_PScov_sim(inp, PS)


## Plots for Power Spectra

### (colored lines show individual simulations, solid black line is mean over simulations, used as our "observation" in LFI or smooth theory curve in the Gaussian likelihood)

In [ ]:
start = 0
if pipeline == 'multifrequency':
    labels = [r'$C_\ell^{00}$', r'$C_\ell^{01}$', r'$C_\ell^{11}$']
else:
    labels = [r'$C_\ell^{\hat{T}\hat{T}}$', r'$C_\ell^{\hat{T}\hat{y}}$', r'$C_\ell^{\hat{y}\hat{y}}$']
fig, axs = plt.subplots(1,3, figsize=(9,5))
ax = 0
for p,q in [(0,0), (0,1), (1,1)]:
    plt.axes(axs[ax])
    for sim in range(len(PS)):
        plt.plot(mean_ells, PS[sim,p,q][start:], alpha=0.2)
    plt.plot(mean_ells, np.mean(PS[:,p,q],axis=0)[start:], color='k', linewidth=1.)
    plt.grid()
    plt.xlabel(r'$\ell$')
    plt.ylabel(labels[ax])
    if p==q:
        plt.yscale('log')
    ax += 1
plt.tight_layout()

## Plot Correlation Matrix and Absolute Value of Covariance Matrix

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (8,4), layout='constrained')
for i in range(2):
    plt.axes(axs[i])
    if i == 0:
        plot_cov_matrix(inp, pipeline, PS, corr=True, colorbar=True, title='Correlation Matrix')
    else:
        plot_cov_matrix(inp, pipeline, PS, corr=False, colorbar=True, title=r'$|$Covariance Matrix$|$')

# Examine Parameter Dependence from Symbolic Regression

### This section won't output anything unless using results from the NILC pipeline or from HILC pipeline with use_symbolic_regression set to True

In [ ]:
b = 5 #bin number (can change this to see parameter dependence in different bins from 0 through Nbins-1)

if (pipeline == 'NILC') or (pipeline == 'HILC' and inp.use_symbolic_regression):
    
    #load file
    fname = f'{inp.output_dir}/data_vecs/best_fits_{naming_str}.p'
    best_fits = pickle.load(open(fname, 'rb'))  
    
    fig, axs = plt.subplots(1,3, figsize=(7,7), sharey=True, sharex=True, layout='constrained')
    axs = axs.flatten()
    npoints = 20
    start_val, end_val = [0.9**2, 1.1**2]
    acmb_test_vals = np.linspace(start_val, end_val, npoints)
    atsz_test_vals = np.linspace(start_val, end_val, npoints)
    comps = ['CMB', 'ftSZ']
    call_fit = lambda A_vec, expr: expr.subs('x0', A_vec[0]).subs('x1', A_vec[1])

    ax = 0
    for p,q in [(0,0), (0,1), (1,1)]:
        plt.axes(axs[ax])
        arr = np.zeros((npoints, npoints))
        for i,acmb in enumerate(acmb_test_vals):
            for j,atsz in enumerate(atsz_test_vals):
                arr[j,i] = call_fit([acmb, atsz], best_fits[p][q][b])

        plt.imshow(arr, extent=[start_val, end_val, start_val, end_val,], vmin=0.85, vmax=1.15, origin='lower')
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.xlabel(r'$A_{\mathrm{CMB}}$', fontsize=16)
        if ax == 0:
            plt.ylabel(r'$A_{\mathrm{ftSZ}}$', fontsize=16)
        ax += 1
        if (p,q) == (0,0):
            title = r' $C_\ell^{\hat{T}\hat{T}}$'
        elif (p,q) == (0,1):
            title = r' $C_\ell^{\hat{T}\hat{y}}$'
        else:
            title = r' $C_\ell^{\hat{y}\hat{y}}$'
        plt.title(title, fontsize=16)

    handles, labels = axs[-1].get_legend_handles_labels() 
    fig.legend(handles, labels, fontsize=10, bbox_to_anchor=(1.0, 0.05), ncol=2)
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=12)
    plt.suptitle(f'Parameter Dependence in Bin {b} (Mean ' + r'$\ell=$' + f'{int(mean_ells[b])})')
    
    print(f'expression for ClTT parameter dependence in bin {b}: ', best_fits[0][0][b])
    print(f'expression for ClTy parameter dependence in bin {b}: ', best_fits[0][1][b])
    print(f'expression for Clyy parameter dependence in bin {b}: ', best_fits[1][1][b])
    
    
    

# Needlet Filters

### This section won't output anything unless pipeline == 'NILC'

In [ ]:
if pipeline == 'NILC':
    ells, h = GaussianNeedlets(inp)
    for n in range(inp.Nscales):
        plt.plot(ells, h[n], color='k')
    plt.grid()
    plt.xlim(0,inp.ellmax)
    plt.xlabel(r'$\ell$')
    plt.ylabel(r'$h_\ell^{(n)}$')